In [15]:
import pandas

import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn

# imported the CSV data
dataframe = pandas.read_csv("Filtered_Data.csv", sep = ",")
dataframe.head()

,cvr_base_data_table.ts,cvr_base_data_table.impression_system_date,cvr_base_data_table.hour,cvr_base_data_table.conversion_status,cvr_base_data_table.cc_id_null_ind,cvr_base_data_table.ad_id,cvr_base_data_table.click_id,cvr_base_data_table.cc_id,cvr_base_data_table.hour_of_day,cvr_base_data_table.raw_publisher_id,...,cvr_base_data_table.click_conf_gmt_offset,cvr_base_data_table.click_state,cvr_base_data_table.click_city,cvr_base_data_table.ad_bid,cvr_base_data_table.ad_cpa_goal,cvr_base_data_table.ad_market_place_id,cvr_base_data_table.click_click_status,cvr_base_data_table.kmean_category_name,cvr_base_data_table.partner_id,cvr_base_data_table.version
0,20190106,2019-01-06,1,1,0,138154,646739319951006038,546739320882001606,20,8CUDMGV55,...,-500,ga,atlanta,0.150000,NaN,1027,1,missing,8PRHGG6T9,1
1,20190106,2019-01-06,1,1,0,138154,646739319951006038,546739320882001606,20,8CUDMGV55,...,-500,ga,atlanta,0.150000,NaN,1027,1,missing,8PRHGG6T9,1
2,20190106,2019-01-06,2,0,0,138154,646740715393002311,346740715723006848,20,8CUDMGV55,...,-600,al,birmingham,0.150000,NaN,1027,1,missing,8PRHGG6T9,1
3,20190106,2019-01-06,4,0,0,138154,246748175931002645,446748176329007771,23,8CUDMGV55,...,-500,in,ft wayne,0.150000,NaN,1027,1,missing,8PRHGG6T9,1
4,20190106,2019-01-06,6,0,0,128699,246754980029002839,646754981260002933,22,8CUX8874N,...,-800,ca,mountain view,3.394224,NaN,1027,1,missing,8PRHGG6T9,1


In [16]:
# removing all data having single values in entire columns
for col in dataframe.columns:
    if(dataframe[col].nunique() == 1):
        dataframe = dataframe.drop(col, axis = 1)

In [17]:
# Remove the timestamp data and impression system date data - results not dependent on the time frame
dataframe = dataframe.drop('cvr_base_data_table.ts', axis = 1)
dataframe = dataframe.drop('cvr_base_data_table.impression_system_date', axis = 1)

# Server hour do not matters
dataframe = dataframe.drop('cvr_base_data_table.hour', axis = 1)

# Data have only 3 values of *au* region, and conversion status 0 for all of them
dataframe = dataframe.drop('cvr_base_data_table.enriched_country', axis = 1)

# Data is similar to cvr_base_data_table.click_city
dataframe = dataframe.drop('cvr_base_data_table.impression_city', axis = 1)

In [18]:
dataframe.head()

,cvr_base_data_table.conversion_status,cvr_base_data_table.ad_id,cvr_base_data_table.click_id,cvr_base_data_table.cc_id,cvr_base_data_table.hour_of_day,cvr_base_data_table.raw_publisher_id,cvr_base_data_table.raw_sub_publisher_id,cvr_base_data_table.raw_sub_sub_publisher_id,cvr_base_data_table.enriched_derived_device,cvr_base_data_table.enriched_derived_os,...,cvr_base_data_table.category_name,cvr_base_data_table.click_browser,cvr_base_data_table.click_user_datetime,cvr_base_data_table.raw_sync_partner_id,cvr_base_data_table.click_conf_gmt_offset,cvr_base_data_table.click_state,cvr_base_data_table.click_city,cvr_base_data_table.ad_bid,cvr_base_data_table.ad_cpa_goal,cvr_base_data_table.partner_id
0,1,138154,646739319951006038,546739320882001606,20,8CUDMGV55,8PO1W66UO,429821366,mobile,android,...,"shopping,consumer,resources,coupons,,,discount...","google,chrome",2019-01-05 20:48:40.883,1,-500,ga,atlanta,0.150000,NaN,8PRHGG6T9
1,1,138154,646739319951006038,546739320882001606,20,8CUDMGV55,8PO1W66UO,429821366,mobile,android,...,"shopping,consumer,resources,coupons,,,discount...","google,chrome",2019-01-05 20:48:40.883,1,-500,ga,atlanta,0.150000,NaN,8PRHGG6T9
2,0,138154,646740715393002311,346740715723006848,20,8CUDMGV55,8PO1W66UO,110622874,mobile,android,...,"shopping,consumer,resources,coupons,,,discount...","google,chrome",2019-01-05 20:11:55.723,1,-600,al,birmingham,0.150000,NaN,8PRHGG6T9
3,0,138154,246748175931002645,446748176329007771,23,8CUDMGV55,8PO1W66UO,110622874,mobile,android,...,"shopping,consumer,resources,coupons,,,discount...","google,chrome",2019-01-05 23:16:16.033,1,-500,in,ft wayne,0.150000,NaN,8PRHGG6T9
4,0,128699,246754980029002839,646754981260002933,22,8CUX8874N,8PO4X3YG2,670653476,mobile,android,...,"finance,grants,,scholarships,,,financial,aid","google,chrome",2019-01-05 22:09:41.026,1,-800,ca,mountain view,3.394224,NaN,8PRHGG6T9
